In [53]:
import numpy as np
import networkx as nx
from copy import deepcopy
import scipy as sp

In [2]:
path = "../OUT/graphs/acc.npz"
edges = np.load(path)["edges"].T
edges.shape

(2283, 2)

In [3]:
G = nx.Graph([tuple(e) for e in edges])

In [4]:
G_adj_matrix = nx.linalg.graphmatrix.adjacency_matrix(G)

In [5]:
A = G_adj_matrix.todense()

In [82]:
n = A.shape[0]
I = np.identity(n)
alpha = 0.02
beta = 1.0
M = I - beta * A 
f = np.matmul(alpha * A, np.ones((n, 1)))
C_katz_ar_our_lstsq, _, _, _ = np.linalg.lstsq(M, f)
C_katz_ar_our_solve = np.linalg.solve(M, f)
# C_katz_ar_our_solve = sp.sparse.linalg.eigsh(M, f)

/var/folders/md/q2tfblls26bglzyrcnsctq_m0000gn/T/ipykernel_9668/3192466613.py:7: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  C_katz_ar_our_lstsq, _, _, _ = np.linalg.lstsq(M, f)


In [83]:
SSP = nx.algorithms.shortest_paths.weighted.single_source_dijkstra_path_length(G, 695)

In [119]:
C_katz = nx.algorithms.centrality.katz_centrality_numpy(G, alpha=0.23, beta=0.85)

In [120]:
C_eig = nx.algorithms.centrality.eigenvector_centrality_numpy(G)

In [121]:
C_pr = nx.algorithms.link_analysis.pagerank_alg.pagerank(G)

In [122]:
C_katz_ar = np.array([C_katz[i] for i in range(len(C_katz))])
C_pr_ar = np.array([C_pr[i] for i in range(len(C_pr))])
C_eig_ar = np.array([C_eig[i] for i in range(len(C_eig))])
SSP_ar = np.array([SSP[i] for i in range(len(SSP))])

In [123]:
C = np.array([C_katz_ar, C_pr_ar, C_eig_ar, np.array(C_katz_ar_our_solve).flatten()])

In [124]:
# fitting
C_fit = np.copy(C)
C_fit[0] = (C_fit[0] - C_fit[0].min()) / (C_fit[0].max() - C_fit[0].min())
C_fit[1] = (C_fit[1] - C_fit[1].min()) / (C_fit[1].max() - C_fit[1].min())
C_fit[2] = (C_fit[2] - C_fit[2].min()) / (C_fit[2].max() - C_fit[2].min())
C_fit[3] = (C_fit[3] - C_fit[3].min()) / (C_fit[3].max() - C_fit[3].min())
SSP_fit = np.copy(SSP_ar)
SSP_fit = (SSP_fit - SSP_fit.min()) / (SSP_fit.max() - SSP_fit.min())

In [125]:
path_out = "../OUT/graphs/acc_cen.npz"
np.savez_compressed(path_out, C=C, C_fit=C_fit, SSP_fit=SSP_fit, SSP=SSP)